In [29]:
import pandas as pd

In [30]:
import os
os.getcwd()

'c:\\Users\\Anton\\Desktop\\s2\\attack-detection-v1'

In [31]:
cases = pd.read_csv("V3_.csv")#pd.read_csv("Validated_V0_Desriptions.csv")

In [32]:
data = pd.read_csv("data/raw_data_sim.csv", parse_dates=["block_time"])

C:\Users\Anton\AppData\Local\Temp\ipykernel_36972\2791140844.py:1: DtypeWarning: Columns (5,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/raw_data_sim.csv", parse_dates=["block_time"])


In [33]:
data.head()

,transaction_id,transaction_time,block_time,sender_id,receiver_id,receiver_type,amount,currency,currency_hash,currency_type,amount_usd,gas_price,gas_limit,gas_used,gas_predicted,balance_in,balance_out,label,risk_level,file_name
0,0x603882c65e180a52ca883dd7817f08d2eb828bbce600...,NaN,2021-09-16 08:45:00,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,NaN,10000.000,HBT,0x0ace32f6e87ac1457a5385f8eb0208f37263b415,erc20,9300.164784,5.370000e-08,412836,283366,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
1,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:00,0xc97f82c80df57c34e84491c0eda050ba924d7429,0xf0a3da0e045fdac0ffec2285d360e706586809e6,NaN,4000.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,4013.530044,5.500000e-08,172764,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
2,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:00,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x3336318c7cd22d0a32477235637f779e7071459b,NaN,2500.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,2508.456277,5.500000e-08,172764,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
3,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:00,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x0b5a2a20ed3773aa33274f6684e97810338c09cb,NaN,2500.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,2508.456277,5.500000e-08,172764,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
4,0x572ccb64ed02bf86d8826a6c47ecbe6fecbe754539c6...,NaN,2021-09-06 10:15:00,0xc97f82c80df57c34e84491c0eda050ba924d7429,0xdf708717070981a8097912318c722f19eefb3bef,NaN,41702.342,HBT,0x0ace32f6e87ac1457a5385f8eb0208f37263b415,erc20,79306.986250,1.400000e-08,103918,68492,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv


In [34]:
df = data.groupby(['file_name'])['label'].sum().reset_index(name = 'n_attacks')  
df.loc[df['n_attacks'] == 1, 'sngl_mlt'] = 'single'
df.loc[df['n_attacks'] > 1, 'sngl_mlt'] = 'multi'
df = df[['file_name' , 'sngl_mlt']]



In [35]:
#case_split = cases.loc[:, ["File Names", "train/test splits"]]
#case_split.columns = ["file_name", "experiment_part"]
#case_split["file_name"] += ".csv"


case_split = cases.loc[:, ["File Names", "train/test splits"]]#cases.loc[:, ["File Names", "train/test splits"]]
case_split.columns = ["file_name", "experiment_part"]
case_split["file_name"] += ".csv"
case_split["file_name"] = 'V3/' + case_split['file_name'].astype(str)#case_split["file_name"] = 'V3/' + case_split['file_name'].astype(str)

In [36]:
data.shape

(951868, 20)

In [37]:
data = data.merge(case_split)

In [38]:
data.shape

(951868, 21)

In [39]:
data = data.merge(df)

In [40]:
data.shape

(951868, 22)

In [41]:
data["experiment_part"].value_counts()

Train         697954
Validation    138322
Test          115587
Fake               5
Name: experiment_part, dtype: int64

In [14]:
data['sngl_mlt'].value_counts()

multi     715841
single    236027
Name: sngl_mlt, dtype: int64

In [15]:
#train_data = data[data['experiment_part'] == 'Train']

## Configure featrure engineering

In [16]:
from cyvers_ai_ds.features import AssetCentricFEPipeline
from cyvers_ai_ds.features.transaction import DEFAULT_AGG_LIST

In [17]:
pipe = AssetCentricFEPipeline()

In [18]:
out_data = pipe.execute(data)

df columns after compute_sender_receiver_info :  Index(['transaction_id', 'transaction_time', 'block_time', 'sender_id',
       'receiver_id', 'receiver_type', 'amount', 'currency', 'currency_hash',
       'currency_type', 'amount_usd', 'gas_price', 'gas_limit', 'gas_used',
       'gas_predicted', 'balance_in', 'balance_out', 'label', 'risk_level',
       'file_name', 'experiment_part', 'sngl_mlt', 'index',
       'snd_rcv_tkn_type_cnt', 'snd_rcv_tx_cnt', 'snd_rcv_amt_usd_sum',
       'snd_rcv_mean_amt_usd', 'snd_rcv_time_diff_sec', 'snd_rcv_min_time',
       'snd_rcv_life_time_sec', 'snd_rcv_mean_time_diff_sec'],
      dtype='object')
df columns after group_by_tx_hash :  Index(['transaction_id', 'sender_id', 'block_time', 'gas_price', 'gas_limit',
       'gas_used', 'snd_rcv_tkn_type_cnt_tx_min',
       'snd_rcv_tkn_type_cnt_tx_max', 'snd_rcv_tkn_type_cnt_tx_median',
       'snd_rcv_tkn_type_cnt_tx_mean', 'snd_rcv_tkn_type_cnt_tx_std',
       'snd_rcv_tkn_type_cnt_tx_sum', 'snd_rcv_tx

In [42]:
tx_df, asset_df = out_data

In [43]:
tx_df.shape

(811733, 114)

In [44]:
tx_df.columns

Index(['transaction_id', 'sender_id', 'block_time', 'gas_price', 'gas_limit',
       'gas_used', 'snd_rcv_tkn_type_cnt_tx_min',
       'snd_rcv_tkn_type_cnt_tx_max', 'snd_rcv_tkn_type_cnt_tx_median',
       'snd_rcv_tkn_type_cnt_tx_mean',
       ...
       'snd_rcv_mean_time_diff_sec_tx_std_log_to_median_ratio',
       'snd_rcv_mean_time_diff_sec_tx_sum_log_to_median_ratio',
       'amount_usd_tx_min_log_to_median_ratio',
       'amount_usd_tx_max_log_to_median_ratio',
       'amount_usd_tx_median_log_to_median_ratio',
       'amount_usd_tx_mean_log_to_median_ratio',
       'amount_usd_tx_std_log_to_median_ratio',
       'amount_usd_tx_sum_log_to_median_ratio',
       'receiver_type_tx_nunique_log_to_median_ratio',
       'internal_tx_cnt_log_to_median_ratio'],
      dtype='object', length=114)

In [45]:
tx_metadata = data.loc[
    :, ["transaction_id", "file_name", "experiment_part", "sngl_mlt"]
].drop_duplicates()

In [46]:
tx_metadata.shape

(813006, 4)

In [47]:
tx_df = tx_df.merge(tx_metadata)

In [48]:
tx_df.shape

(813006, 117)

In [26]:
#tx_df = tx_df[tx_df["experiment_part"] == 'Train']

In [49]:
tx_df["experiment_part"].value_counts()


Train         651040
Validation    112755
Test           49206
Fake               5
Name: experiment_part, dtype: int64

In [50]:
tx_df["sngl_mlt"].value_counts()


multi     614857
single    198149
Name: sngl_mlt, dtype: int64

,transaction_id,sender_id,block_time,gas_price,gas_limit,gas_used,snd_rcv_tkn_type_cnt_tx_min,snd_rcv_tkn_type_cnt_tx_max,snd_rcv_tkn_type_cnt_tx_median,snd_rcv_tkn_type_cnt_tx_mean,...,amount_usd_tx_max_log_to_median_ratio,amount_usd_tx_median_log_to_median_ratio,amount_usd_tx_mean_log_to_median_ratio,amount_usd_tx_std_log_to_median_ratio,amount_usd_tx_sum_log_to_median_ratio,receiver_type_tx_nunique_log_to_median_ratio,internal_tx_cnt_log_to_median_ratio,file_name,experiment_part,sngl_mlt
640297,0xfe4589d222b704a9b9ed180b5acffa81693cd9f9fa56...,0xbea134b1920311f241416ba3bc6ebac277724d8a,2022-05-11 11:54:00,2.000000e-08,125000,21051,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,V3/fake_file3.csv,Fake,multi
640299,0x8fab10f1132369313063310d636b3b3f12ac975703bb...,0xbea134b1920311f241416ba3bc6ebac277724d8a,2022-05-11 14:44:00,2.000000e-08,125000,21000,1.0,1.0,1.0,1.0,...,-0.190505,-0.190505,-0.190505,0.0,-0.190505,0.0,0.0,V3/fake_file3.csv,Fake,multi
640301,0x34ee2879f360010e7d496ba30b88d5229e983080dd53...,0xbea134b1920311f241416ba3bc6ebac277724d8a,2022-05-11 19:18:00,2.000000e-08,250000,21000,1.0,1.0,1.0,1.0,...,-5.959458,-5.959458,-5.959458,0.0,-5.959458,0.0,0.0,V3/fake_file3.csv,Fake,multi
640303,0xd1b809ca1a4b221377de2de0c77a6d028c9b4756dc70...,0xbea134b1920311f241416ba3bc6ebac277724d8a,2022-05-11 19:18:00,2.000000e-08,250000,63197,1.0,1.0,1.0,1.0,...,-2.798592,-2.798592,-2.798592,0.0,-2.798592,0.0,0.0,V3/fake_file3.csv,Fake,multi
640305,0x5e572187eb87d7b16f73b6be8b8d4ea06d53b1106ba1...,0xbea134b1920311f241416ba3bc6ebac277724d8a,2022-05-11 19:19:00,2.000000e-08,250000,65613,2.0,3.0,2.5,2.5,...,-0.000103,-0.000103,-0.000103,0.0,-0.000103,0.0,0.0,V3/fake_file3.csv,Fake,multi


In [ ]:
#tx_df.loc[tx_df["experiment_part"] == "Train", :].to_csv(
#    "data/train_data.csv", index=False
#)

In [52]:
tx_df.loc[tx_df["experiment_part"] == "Train", :].to_csv(
    "data/train_data.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Validation", :].to_csv(
    "data/validation_data.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Test", :].to_csv(
    "data/test_data.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Fake", :].to_csv(
    "data/fake_data.csv", index=False
)

Back up data to s3

In [ ]:
from utils import MODEL_NAME
from utils.s3 import MODELS_BUCKET
import boto3

In [ ]:
bucket = boto3.resource('s3').Bucket(MODELS_BUCKET)
bucket.upload_file('data/train_data.csv', MODEL_NAME + '/data/train_data.csv')
bucket.upload_file('data/validation_data.csv', MODEL_NAME + '/data/validation_data.csv')
bucket.upload_file('data/test_data.csv', MODEL_NAME + '/data/test_data.csv')